In [2]:
import pandas as pd
import os
import numpy as np

# Extracción de datos de Maps

In [5]:
# Script para cargar los metadatos de maps.

maps_metadata = pd.DataFrame()
lista_metadata = []
for i in range(1,3):
    df = pd.read_json(f'./data/Google Maps/metadata-sitios/{i}.json',lines=True)
    maps_metadata = pd.concat([maps_metadata,df])
maps_metadata = maps_metadata.reset_index(drop=True)
for i in range(len(maps_metadata)):
    if type(maps_metadata.name[i]) != str:
        lista_metadata.append(i)
maps_metadata.drop(lista_metadata,axis=0,inplace=True)
horarios_maps = maps_metadata[['gmap_id','hours']]
categorias_maps = maps_metadata[['gmap_id','category']]
servicios_maps = maps_metadata[['gmap_id','MISC']]
relativos_maps = maps_metadata[['gmap_id','relative_results']]
maps_metadata = maps_metadata[['gmap_id','name','address','avg_rating','num_of_reviews','price']]


### Script para encontrar ids únicos y después utilizarlos para filtrar el dataframe de reviews con el fin de obtener los establecimientos del giro que deseamos.

categorias = categorias_maps.category.apply(pd.Series,dtype=str)
cols = categorias.shape[1]
lista_cols =[]
for i in range(1,cols+1):
    lista_cols.append(f'cat_{i}')
categorias.columns = lista_cols
cat = pd.concat([categorias_maps['gmap_id'],categorias],axis=1)
cat = pd.melt(cat, id_vars='gmap_id', value_vars=lista_cols, value_name='Categoria')[['gmap_id','Categoria']].dropna().reset_index(drop=True)
# Filtros para locales de alimentos en general.
restaurant = cat.Categoria.str.contains('restaurant') | cat.Categoria.str.contains('Restaurant')
diner = cat.Categoria.str.contains('Diner') | cat.Categoria.str.contains('diner')
grill = cat.Categoria.str.contains('grill') | cat.Categoria.str.contains('Grill')
eatery = cat.Categoria.str.contains('grill') | cat.Categoria.str.contains('Grill')
pizza = cat.Categoria.str.contains('pizzeria') | cat.Categoria.str.contains('Pizzeria') | cat.Categoria.str.contains('Pizza')
tea = cat.Categoria.str.contains('tea') | cat.Categoria.str.contains('Tea')
coffee = cat.Categoria.str.contains('coffee') | cat.Categoria.str.contains('Coffee')
lunch = cat.Categoria.str.contains('lunch') | cat.Categoria.str.contains('Lunch') | cat.Categoria.str.contains('Brunch')
steak = cat.Categoria.str.contains('steak') | cat.Categoria.str.contains('Steak')
snack = cat.Categoria.str.contains('snack') | cat.Categoria.str.contains('Snack')
others_0 = cat.Categoria.str.contains('breakfast') | cat.Categoria.str.contains('Breakfast') | cat.Categoria.str.contains('fast food') | cat.Categoria.str.contains('Fast food')
others_1 = cat.Categoria.str.contains('dinner') | cat.Categoria.str.contains('Dinner')  | cat.Categoria.str.contains('buffet') | cat.Categoria.str.contains('Buffet') | cat.Categoria.str.contains('cafe') | cat.Categoria.str.contains('Cafe') | cat.Categoria.str.contains('seafood ') | cat.Categoria.str.contains('Seafood ') | cat.Categoria.str.contains('health food') | cat.Categoria.str.contains('Health food') | cat.Categoria.str.contains('ramen') | cat.Categoria.str.contains('Ramen') 
food = cat.Categoria.str.contains('food') | cat.Categoria.str.contains('Food')
# Obteniendo los id's únicos para todos los filtros anteriores.
cat = cat[ restaurant | diner | grill | eatery | pizza | tea | coffee | lunch | steak | snack | others_0 | others_1 | food].reset_index(drop=True)
lista_ids = pd.unique(cat.gmap_id).tolist()
cat[cat.isin({'gmap_id':lista_ids})].gmap_id.tolist()


# Aquí cargamos las reviews.

source_path = './data/Google Maps/estados'
list_state = os.listdir(f'{source_path}')
maps_reviews = pd.DataFrame()
for i in list_state:
    list_ = os.listdir(f'{source_path}/{i}')
    for j in list_:
        df = pd.read_json(f'{source_path}/{i}/{j}',lines=True)
        df['estado'] = i[7:]
        maps_reviews = pd.concat([maps_reviews,df],axis=0)
    break
maps_reviews = maps_reviews[['gmap_id','time','name','estado','rating','text','resp']].reset_index(drop=True)
maps_reviews_clean = maps_reviews[maps_reviews.gmap_id.isin(lista_ids)]


# Script para joinear la tabla de hechos (reviews) con la de dimensiones (metadata). Ya están limpias ambas tablas, considerando solo el giro que queremos abordar.

maps_data = maps_reviews_clean.merge(cat,on='gmap_id',how='left')
maps_data = maps_reviews.merge(maps_metadata, on='gmap_id')
maps_data.columns = ['gmap_id','time','user_name','estado','rating','text','resp','business_name','address','avg_rating','num_of_reviews','price']
maps_data.user_name = maps_data.user_name.astype(str)
maps_data.text = maps_data.text.astype(str)
maps_data.time = maps_data.time.astype(np.int64)
maps_data.gmap_id = maps_data.gmap_id.astype(str)

maps_data['resp_time'] = pd.Series(dtype=np.int64)
maps_data['resp_text'] = pd.Series(dtype=str)
for i in range(len(df)):
    if type(maps_data.resp[i]) == dict:
        maps_data.loc[i,'resp_time'] = maps_data.resp[i]['time']
        maps_data.loc[i,'resp_text'] = maps_data.resp[i]['text']
    else:
        maps_data.loc[i,'resp_time'] = 0
        maps_data.loc[i,'resp_text'] = ''
maps_data.resp_time = maps_data.resp_time.fillna(0).apply(lambda x: int(x))
maps_data.resp_text = maps_data.resp_text.fillna('')
maps_data = maps_data[['gmap_id','time','user_name','estado','rating','text','business_name','address','avg_rating','num_of_reviews','price','resp_time','resp_text']]
maps_data = maps_data[maps_data.duplicated() == False].reset_index(drop=True)

maps_data.head()

,gmap_id,time,user_name,estado,rating,text,business_name,address,avg_rating,num_of_reviews,price,resp_time,resp_text
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,1609909927056,Song Ro,California,5,Love there korean rice cake.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",4.4,18,None,0,
1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,1612849648663,Rafa Robles,California,5,Good very good,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",4.4,18,None,0,
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,1583643882296,David Han,California,4,They make Korean traditional food very properly.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",4.4,18,None,0,
3,0x80c2c778e3b73d33:0xbdc58662a4a97d49,1551938216355,Anthony Kim,California,5,Short ribs are very delicious.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",4.4,18,None,0,
4,0x80c2c778e3b73d33:0xbdc58662a4a97d49,1494910901933,Mario Marzouk,California,5,Great food and prices the portions are large,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",4.4,18,None,0,


In [ ]:
lista_idx = []
for i in range(len(maps_data)):
    if type(maps_data.price[i]) != type(None):
        lista_idx.append(i)
maps_data_wprice = maps_data.loc[lista_idx]
maps_data_wprice = maps_data_wprice[maps_data_wprice.duplicated() == False].reset_index(drop=True)
maps_data_wprice.shape

(24542, 12)

In [ ]:
lista_idx = []
for i in range(len(maps_data)):
    if maps_data.resp_text[i] != '':
        lista_idx.append(i)
maps_data_wresp = maps_data.loc[lista_idx]
maps_data_wresp = maps_data_wresp[maps_data_wresp.duplicated() == False].reset_index(drop=True)
maps_data_wresp.shape

(12139, 12)

In [ ]:
horarios_maps.head()

,gmap_id,hours
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun..."
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],..."
3,0x80c2c89923b27a41:0x32041559418d447,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu..."
4,0x80c2c632f933b073:0xc31785961fe826a6,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu..."


In [10]:
categorias_maps.head()

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,[Pharmacy]
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,[Textile exporter]
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant]
3,0x80c2c89923b27a41:0x32041559418d447,[Fabric store]
4,0x80c2c632f933b073:0xc31785961fe826a6,[Fabric store]


In [11]:
relativos_maps.head()

,gmap_id,relative_results
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1..."
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2..."
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2..."
3,0x80c2c89923b27a41:0x32041559418d447,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2..."
4,0x80c2c632f933b073:0xc31785961fe826a6,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2..."


In [12]:
servicios_maps.head()

,gmap_id,MISC
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"{'Service options': ['In-store shopping', 'Sam..."
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"{'Service options': ['Takeout', 'Dine-in', 'De..."
3,0x80c2c89923b27a41:0x32041559418d447,"{'Service options': ['In-store shopping'], 'Pa..."
4,0x80c2c632f933b073:0xc31785961fe826a6,{'Service options': ['In-store pickup']}


### Data Cleaning for one file of one state of maps data

In [ ]:
df = pd.read_json('data/Google Maps/estados/review-Texas/1.json', lines=True)
df.user_id = df.user_id.astype(str)
df.user_id = df.user_id.apply(lambda x: x.replace('e+20','').replace('.',''))
df.user_id = df.user_id.apply(lambda x: int(x))
df.name = df.name.astype(str)
df.text = df.text.astype(str)
df.pics = df.pics.astype(str)
df.gmap_id = df.gmap_id.astype(str)

df['resp_time'] = pd.Series()
df['resp_text'] = pd.Series()
for i in range(len(df)):
    # print(i)
    if type(df.resp[i]) == dict:
        df.loc[i,'resp_time'] = df.resp[i]['time']
        df.loc[i,'resp_text'] = df.resp[i]['text']
    else:
        df.loc[i,'resp_time'] = 1
        df.loc[i,'resp_text'] = ''
df.resp_time = df.resp_time.fillna(0).apply(lambda x: int(x))
df.resp_text = df.resp_text.fillna('')
df = df[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id', 'resp_time', 'resp_text']]

ndjson = df.to_json(orient='records',lines=True) # 'data/Google Maps/estados/review-Texas/1_test.json',
with open('data/Google Maps/estados/review-Texas/1_test.ndjson','w') as f:
    f.write(ndjson)

# Extracción de datos de Yelp

In [1]:
dfy_rev = pd.read_json('data/Yelp/review.json',lines=True,nrows=100000)['review_id','user_id','business_id','stars','text','date']
dfy_rev

NameError: name 'pd' is not defined

In [7]:
dfy_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [9]:
dfy_rev.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [10]:
dfy_rev.describe()

,stars,useful,funny,cool
count,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06
mean,3.748584e+00,1.184609e+00,3.265596e-01,4.986175e-01
std,1.478705e+00,3.253767e+00,1.688729e+00,2.172460e+00
min,1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,5.000000e+00,1.182000e+03,7.920000e+02,4.040000e+02


In [5]:
dfy_user = pd.read_parquet(r'C:\Users\tinma\OneDrive\Escritorio\Yelp\user.parquet')
dfy_use

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,74,9,6,,kmwNG5LZSHFmveg6wYYdrw,0,...,1,0,0,0,1,4,0,0,1,0
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,14,5,2,,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",1,...,1,0,0,0,0,1,0,0,0,0
2105594,tpBznnD6uJN3m_pJubj09w,Emily,26,2013-08-13 23:18:11,4,1,2,,"bKV3ly2MuK-K1cptMrFknQ, liel18zRoSB4tEkUP7i6Cg...",0,...,0,0,0,0,1,0,0,0,0,0
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
df_tip = pd.read_json(r'C:\Users\tinma\OneDrive\Escritorio\Yelp\tip.json',lines=True)
df_tip

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [22]:
dfy_bsi = pd.read_pickle(r'C:\Users\tinma\OneDrive\Escritorio\Yelp\business.pkl')
dfy_bsi = dfy_bsi.iloc[:,:14]
dfy_chip = dfy_bsi[dfy_bsi.name=='Chipotle']
dfy_chip

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
115794,1MVoJvPhP5SkrJGCxuriaQ,Chipotle,"1851 S Columbus Blvd, Ste 3",Philadelphia,TN,19148,39.923491,-75.140583,1.0,8,1,"{'RestaurantsDelivery': 'True', 'RestaurantsTa...","Fast Food, Restaurants, Mexican","{'Monday': '0:0-0:0', 'Tuesday': '10:45-22:0',..."


In [23]:
dfy_bsi[dfy_bsi.name == 'McDonalds']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
28756,2d7rO6a0HdyEg2Nnf3kLKg,McDonalds,11240 82 Street NW,Edmonton,PA,T5B 0C1,53.561686,-113.466926,2.0,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Restaurants, Food, Fast Food",None
91266,A1hHP1HZfebuI9-sdieZBQ,McDonalds,34 Avenue 99 Street,Edmonton,TN,T5V 1T8,53.468252,-113.486935,2.0,14,1,"{'HasTV': 'True', 'GoodForKids': 'True', 'Rest...","Fast Food, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
102306,ln57otfbTmN8FsLlR4x57Q,McDonalds,"30th St Station, 2951 Market St",Philadelphia,FL,19104,39.961793,-75.19721,2.5,10,0,"{'RestaurantsReservations': 'False', 'Restaura...","Burgers, Restaurants, Fast Food",None
103477,jDGxzwlkyH3I7_LghcxLAA,McDonalds,Philadelphia Airport Terminal E,Philadelphia,ID,19153,39.877906,-75.236665,2.0,12,0,"{'WiFi': ''no'', 'NoiseLevel': ''average'', 'B...","Burgers, Fast Food, Restaurants",None
128629,FKwQsrcZXy-NUKszOcV8mQ,McDonalds,934 Canal St,New Orleans,NV,70112,29.954638,-90.071902,2.0,10,0,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Restaurants, Fast Food",None


In [2]:
'review-New_York'[7:]

'New_York'